In [0]:
# Load it into a DataFrame
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True, inferSchema=True)
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
df.count()

Out[5]: 294

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
top_10_city=df_selected.orderBy("2014 Population estimate", ascending=False).limit(10)
display(top_10_city)

City,State,State Code,2014 Population estimate,2015 median sales price
New York[6],New York,NY,8491079,388.6
Los Angeles,California,CA,3928864,434.7
Chicago,Illinois,IL,2722389,192.5
Houston[7],Texas,TX,2239558,200.3
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Phoenix,Arizona,AZ,1537058,206.1
San Antonio,Texas,TX,1436697,184.7
San Diego,California,CA,1381069,510.3
Dallas,Texas,TX,1281047,192.5
San Jose,California,CA,1015785,900.0


Databricks visualization. Run in Databricks to view.

SCENARIO 2 : Average Median Sales price by state

In [0]:
from pyspark.sql.functions import avg
df_average=df_selected.groupBy("State").agg(avg("2015 median sales price"))
display(df_average)

State,avg(2015 median sales price)
Utah,243.3
Hawai'i,699.3
Minnesota,209.4
Ohio,112.86666666666667
Arkansas,131.8
Oregon,231.53333333333333
Texas,173.4777777777778
North Dakota,180.0
Pennsylvania,183.8
Connecticut,250.13333333333333


Databricks visualization. Run in Databricks to view.

FILTER THE CITY WITH POPULATION > 1 Milllion

In [0]:
from pyspark.sql.functions import col
cities_over_million = df.filter(df_selected["2014 Population estimate"] > 1_000_000)
display(cities_over_million)


2014 rank,City,State,State Code,2014 Population estimate,2015 median sales price
6,Phoenix,Arizona,AZ,1537058,206.1
2,Los Angeles,California,CA,3928864,434.7
8,San Diego,California,CA,1381069,510.3
10,San Jose,California,CA,1015785,900.0
3,Chicago,Illinois,IL,2722389,192.5
1,New York[6],New York,NY,8491079,388.6
5,Philadelphia[8],Pennsylvania,PA,1560297,204.9
9,Dallas,Texas,TX,1281047,192.5
4,Houston[7],Texas,TX,2239558,200.3
7,San Antonio,Texas,TX,1436697,184.7


Databricks visualization. Run in Databricks to view.

IDENTIFYING CITIES WITH MISSING OR ZERO PRICE VALUE

In [0]:
df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"] == 0)).show()

+-------------+----------+----------+------------------------+-----------------------+
|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+-------------+----------+----------+------------------------+-----------------------+
|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|     Chandler|   Arizona|        AZ|                  254276|                   null|
|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|     Glendale|   Arizona|        AZ|                  237517|                   null|
|         Mesa|   Arizona|        AZ|                  464704|                   null|
|       Peoria|   Arizona|        AZ|                  166934|                   null|
|   Scottsdale|   Arizona|        AZ|                  230512|                   null|
|     Surprise|   Arizona|        AZ|                  126275|                   null|
|        Tempe|   Arizona|        AZ|      

In [0]:
|